Making a copy of dimensions.date_dimension in the  MMAI db in my personal db

In [9]:
#import libraries 
import sqlalchemy as sa
from sqlalchemy.schema import ForeignKeyConstraint, MetaData, Table
import requests,json
import pandas as pd

In [1]:
#create an engine to access the mmaidb 
mmai_db_secret = {

    'drivername' : 'postgresql+psycopg2',
    'host'       : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'       : '5432',
    'username'   : 'pelumi',
    'password'   : '2023!Schulich',
    'database'   : 'mmai_db'

}


In [2]:
#create an engine to access pelumidb
pelumi_db_secret = {

    'drivername' : 'postgresql+psycopg2',
    'host'       : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'       : '5432',
    'username'   : 'pelumi',
    'password'   : '2023!Schulich',
    'database'   : 'pelumi_db'

}

In [4]:
#create mmai database url 
mmai_db_url = sa.engine.URL.create(
    drivername = mmai_db_secret['drivername'],
    username   = mmai_db_secret['username'],
    password   = mmai_db_secret['password'],
    host       = mmai_db_secret['host'],
    port       = mmai_db_secret['port'],
    database   = mmai_db_secret['database']
)

In [12]:
#create mmai engine 
mmai_db_engine = sa.create_engine(mmai_db_url)

In [13]:
#pull data from mmai engine
mmai_con = mmai_db_engine.connect()


In [14]:
#read data from mmaidb
with mmai_db_engine.connect() as connection:
    date_dimensions = pd.read_sql(
        sql='SELECT * FROM dimensions.date_dimension',
        con = connection
    )

date_dimensions

,sk_date,date,day_name,day_of_month,day_of_year,month,month_name,year,year_week,week,running_week,year_quarter,quarter,running_quarter
0,20190101,2019-01-01,Tuesday,1,1,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
1,20190102,2019-01-02,Wednesday,2,2,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
2,20190103,2019-01-03,Thursday,3,3,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
3,20190104,2019-01-04,Friday,4,4,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
4,20190105,2019-01-05,Saturday,5,5,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,20231227,2023-12-27,Wednesday,27,361,12,December,2023,2023-W52,W52,261,2023-Q4,Q4,20
1822,20231228,2023-12-28,Thursday,28,362,12,December,2023,2023-W52,W52,261,2023-Q4,Q4,20
1823,20231229,2023-12-29,Friday,29,363,12,December,2023,2023-W52,W52,261,2023-Q4,Q4,20
1824,20231230,2023-12-30,Saturday,30,364,12,December,2023,2023-W52,W52,261,2023-Q4,Q4,20


In [26]:

#create database pelumidb url 
pelumi_db_url = sa.engine.URL.create(
    drivername = pelumi_db_secret['drivername'],
    username   = pelumi_db_secret['username'],
    password   = pelumi_db_secret['password'],
    host       = pelumi_db_secret['host'],
    port       = pelumi_db_secret['port'],
    database   = pelumi_db_secret['database']
)

In [27]:
#create pelumidb engine 
pelumi_db_engine = sa.create_engine(pelumi_db_url)

In [28]:
#create schema
qwhere = '''
CREATE SCHEMA IF NOT EXISTS final_exam;
'''
with pelumi_db_engine.connect() as connection:
    return_query = connection.execute(qwhere)

In [25]:
#push date dimensions date to Pelumidb
#pandas dataframe to sql
date_dimensions.to_sql(
name = 'date_dimensions',
con = pelumi_db_engine,
schema= 'final_exam',
if_exists ='replace',
index = False,
dtype = {'sk_date':sa.types.INTEGER, 
         'date':sa.types.DATE, 
         'day_name':sa.types.VARCHAR, 
         'day_of_month':sa.types.INTEGER, 
         'day_of_year':sa.types.INTEGER, 
         'month':sa.types.INTEGER,
         'month_name':sa.types.VARCHAR, 
         'year':sa.types.INTEGER, 
         'year_week':sa.types.VARCHAR, 
         'week':sa.types.VARCHAR, 
         'running_week':sa.types.INTEGER,
         'year_quarter':sa.types.VARCHAR, 
         'quarter':sa.types.VARCHAR, 
         'running_quarter':sa.types.INTEGER},
method = 'multi'
)